In [ ]:
import os
from datetime import datetime
import numpy as np

In [ ]:
# local libraries
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model
from coupling_PCR_FM import configuration

In [ ]:
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# -------------------------------------------------------------------------------------------------
# IMPORT MODEL SETTINGS FROM INI-FILE
# -------------------------------------------------------------------------------------------------

# parse set/ini-file with central/general settings for coupling framework
config = configuration.Configuration()
config.parse_configuration_file(argv1)

# parse env-file for user-specific paths and environmental variables
envs = configuration.Configuration()
envs.parse_configuration_file(argv2)

options = {section: eval('config.{:s}'.format(section)) for section in config.allSections}
options.update({section: eval('envs.{:s}'.format(section)) for section in envs.allSections})
start_date = datetime.strptime(options['numerical_settings']['startTime'], '%Y-%m-%d')
end_date = datetime.strptime(options['numerical_settings']['endTime'], '%Y-%m-%d')
options

In [ ]:
# setup output dir
cwd = os.getcwd() # note: this get changed by pcr initialization later on
out_dir = envs.PCRpaths['outputDirectoryPCR']
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
print(out_dir)
print(cwd)

In [ ]:
# initialize PCR model
PCR_configfile = os.path.join(cwd, options['hydrologic_model']['config_dir'], options['hydrologic_model']['config_file'])
PCR_in_dir = options['PCRpaths']['inputDirectoryPCR']
PCR_out_dir = os.path.join(out_dir, 'PCR')

hydrologicModel = PCR_model()
hydrologicModel.initialize_offline_forcing(PCR_configfile, start_date, end_date,
                                           out_dir = PCR_out_dir, in_dir = PCR_in_dir)

In [ ]:
# initialize CMF model
CMF_engine = os.path.join(cwd, options['CMF_engine']['CMF_path'])
CMF_model_dir = os.path.join(cwd, options['routing_model']['model_dir'])
CMF_config_fn = os.path.join(CMF_model_dir, options['routing_model']['model_file'])
CMF_out_dir = os.path.join(out_dir, 'CMF')

routingModel = CMF_model(CMF_engine)
routingModel.initialize_online_forcing(CMF_config_fn, CMF_model_dir,
                                       start_date, end_date, 
                                       out_dir = CMF_out_dir)

In [ ]:
# set coupling function

def PCR2CMF_runoff(PCR_BMI, CMF_bmi):
    "creates a 'get_runoff' function to get runoff from PCR model fit for CMF model"
    def update_states():
        "coupling runoff between CMFan PCR model"
        runoff = PCR_BMI.get_var('landSurfaceRunoff')
        runoff = np.where(np.isnan(runoff), 0, runoff)
        CMF_bmi.set_var("runoff", runoff)
    return update_states

CFM_get_runoff = PCR2CMF_runoff(hydrologicModel, routingModel)
routingModel.set_update_states(CFM_get_runoff)

In [ ]:
# run coupled model for 20 timesteps
for i in range(20):
    hydrologicModel.update()
    routingModel.update_states()
    routingModel.update()   

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# hydrologicModel.update()
# routingModel.update_states()
# routingModel.update() 

runoff = hydrologicModel.get_var('landSurfaceRunoff')
outflw = routingModel.get_var('outflw')
outflw = np.ma.masked_greater(outflw, 1.e19)
rivsto = routingModel.get_var('rivsto_out')
rivsto = np.ma.masked_greater(rivsto, 1.e19)

fig = plt.figure(figsize=(15, 5))
a=fig.add_subplot(1,3,1)
imgplot = plt.imshow(runoff)
a.set_title('PCR runoff')
plt.colorbar(orientation ='horizontal')
a=fig.add_subplot(1,3,2)
imgplot = plt.imshow(outflw)
a.set_title('CMF outflw')
plt.colorbar(orientation='horizontal')
a=fig.add_subplot(1,3,3)
imgplot = plt.imshow(rivsto)
a.set_title('CMF rivsto')
plt.colorbar(orientation='horizontal')